# Version 4 

L'objectif de cette version est d'utiliser les données historiques et le nouveau cadre de simulation fait en version 3 pour pouvoir implémenter :
1. Un algo ultra basique comme fait en version 2
2. Un algo un peu plus élaboré basé sur le principe du filtre de Kalman

Questionnement :
1. Pas d'interet de faire tourner l'ancien algo car il va forcément bien marcher même sur le nouveau cadre étant donnée que le nombre de seconde dans une journée est connue (horaires fixes d'ouverture des br).
2. Si jamais je devais le faire tourner, ça serait sur les données historiques générées ou faire l'implémentation "en temps réel" en parallèle de la simulation ? 

### Etape 1 - Reprendre le code propre de simulation

In [2]:
import numpy as np
import random 
import simpy
import time
import csv
from collections import namedtuple
from datetime import datetime
import pandas as pd

In [3]:
def panneaux_dispo(nb_heures, total_panneaux):
    prop = np.random.normal(loc=0.8, scale=0.2, size=nb_heures)
    pann_dispo = list(map(lambda x: int(total_panneaux*x if x<=1 else total_panneaux), prop))
    return pann_dispo

In [4]:
def lambda_br(pann_dispo):
    lam = list(map(lambda x: int((1000-x)/50) if x<950 else 1, pann_dispo))
    return lam

In [5]:
def imps():
    lam = int(np.random.normal(loc=4, scale=2, size=1))
    if lam < 1:
        lam = 1
    nb_imp = np.random.poisson(lam)
    return nb_imp

In [6]:
def delai(lam):
    secondes = np.random.poisson(lam)
    
    #Simuler une proba de pb technique (1% de chance d'en avoir un)
    if not random.random() < 0.99:
        secondes = np.random.poisson(lam + 1000)
    return secondes

In [7]:
def total_sec(nb_jours):
    tot = nb_jours * 86400
    return tot

In [8]:
def sauvegarde(liste, nom_de_fichier):
    with open(nom_de_fichier, "w", encoding="utf8") as fichier:
        #Recuperer le nom des colonnes pour la première ligne
        premier, *_ = liste
        ecrivain = csv.DictWriter(fichier, premier._fields)
        ecrivain.writeheader()
        for br in liste:
            # On ecrit chaque ligne comme un ordered dict
            ecrivain.writerow(br._asdict())

In [9]:
def open_rtb(env, P, timestampnow, nb_jours, bidrequests, data):
    dispo = panneaux_dispo(14, 1000)
    lam_secondes = lambda_br(dispo)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
    i = 0
    identifiant = 0
    while True:
        # Déterminer dans quel état on est actuellement
        if current_hour >= 6 and current_hour < 20:
            if setup:
                setup = False
                dispo = panneaux_dispo(14, 1000)
                lam_secondes = lambda_br(dispo)
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
                i = 0
            
            # Génération d'une br
            identifiant += 1
            rt = (timestampnow+total_sec(nb_jours))-env.now

            #Timestamp de la br
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")

            # Nombre d'impressions
            nb_imp = imps()
            Prix = P * nb_imp

             # Stocker les résultats dans le namedtuple puis ajout à une liste
            resultats = bidrequests(
                identifiant = identifiant,
                timestamp = env.now,
                timestamp_string = time,
                nombre_impressions = nb_imp,
                prix_impression = P,
                prix_total = Prix
            )
            data.append(resultats)

            # Detecter quelle heure est-t-il
            if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour != current_hour:
                i += 1 
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour

            # Temps avant la prochaine BR
            # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
            try:
                time_before_next = delai(lam_secondes[i])
            except IndexError:
                time_before_next = delai(lam_secondes[i-1])


            # Fin de la simulation
            if rt < time_before_next:
                print(f"Fin de la simulation à {time} !")

            yield env.timeout(time_before_next)
            
        else:
            setup = True
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
            rt = (timestampnow+total_sec(nb_jours))-env.now
            if rt <= 1:
                print(f"Fin de la simulation à {time} !")
            yield env.timeout(1)  

In [9]:
# Constante 
nb_days = 30
prix_per_imp = 1

In [10]:
timestampnow = int(time.time()-3600*8)
# Générer un named tuple pour stocker les résultats
bidrequests =  namedtuple(
    "bidrequests", 
    (
        "identifiant", 
        "timestamp", 
        "timestamp_string",
        "nombre_impressions",
        "prix_impression",
        "prix_total"
    )
                    )
# Liste pour stocker les résultats du namedtuple
data = list()
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env, prix_per_imp, timestampnow, nb_days, bidrequests, data))

In [11]:
env.run(until=timestampnow + total_sec(nb_days))

Fin de la simulation à 07-12-2020 03:05:59 !


In [12]:
len(data)

112398

# Algorithme de pacing uniforme 

Pour générer une dépense lissée sur toute la journée, une solution pourrait être de se ramener à un problème d'optimisation sous contrainte. Seulement, dans le cadre du DOOH (enfin il me semble), chaque impression n'est pas quantifiable en terme de qualité. On peut simplement acquérir une impression mais on ne sais pas si c'est une impression de bonne qualité étant donné que nous n'avons pas de données telles que la probabilité de conversion, le teaux de clic, le taux d'action etc etc. 

L'idée est donc de simplement dans un premier temps avoir un algorithme qui fonctionne automatiquement qui a pour but d'acquérir des impressions toute la journée sans générer de "pics" ou de "creux" de dépense (hors problème technique par exemple).

L'algorithme pourrait donc estimer les horaires à laquelle on reçoit des bids requests et selon ces horaires découper la journée en $t$ time slots (en secondes voire encore plus précis) et donc assigner un budget par time slot. 

$$b_{t+1} = \Bigr(B - \sum_{s=1}^{t} S(s)\Bigl) \dfrac{1}{T-t}$$

où $b_{t+1}$ est le budget à allouer à la seconde $t+1$, $B$ le budget total de la journée, $S(s)$ est la dépense effective à la seconde $t$ et enfin $T-t$ représente le temps restant en secondes jusqu'à la fin de la journée.

### On modifie la fonction qui génère les données pour inclure le petit programme de pacing

In [54]:
def open_rtb_pacing(env, P, B, timestampnow, nb_jours, nb_hours_per_day, bidrequests, data):
    setup = True
    identifiant = 0
    cible = B/(nb_hours_per_day*3600)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
    while True:
        # Déterminer dans quel état on est actuellement
        if current_hour >= 6 and current_hour < 20:
            if setup:
                setup = False
                dispo = panneaux_dispo(nb_hours_per_day, 1000)
                lam_secondes = lambda_br(dispo)
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
                i = 0
                S = list()
                end_day = env.now + (nb_hours_per_day*3600) - 1200 #On fixe la fin de la journée à 19h40h pour être plus restrictif
            
            # Génération d'une br
            identifiant += 1
            rt = (timestampnow+total_sec(nb_jours))-env.now
            
            # Déterminer b_t
            try:
                bt = (B - sum(S))/(end_day - env.now)
                
            except ZeroDivisionError:
                bt = 1
            #on force l'achat
            if bt < 0:
                bt=1
            
            #Timestamp de la br
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")

            # Nombre d'impressions
            nb_imp = imps()
            Prix = P * nb_imp
            
            #Achat ou non 
            if bt >= cible and (sum(S)+Prix) <= B:
                S.append(Prix)
                achat = 1
            else:
                achat = 0

            # Stocker les résultats dans le namedtuple puis ajout à une liste
            resultats = bidrequests(
                identifiant = identifiant,
                timestamp = env.now,
                timestamp_string = time,
                nombre_impressions = nb_imp,
                prix_impression = P,
                prix_total = Prix,
                achat = achat,
                budget_depense = sum(S),
                bt = bt,
                cible = cible
            )
            data.append(resultats)

            # Temps avant la prochaine BR
            # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
            try:
                time_before_next = delai(lam_secondes[i])
            except IndexError:
                time_before_next = delai(lam_secondes[i-1])


            # Fin de la simulation
            if rt < time_before_next:
                print(f"Fin de la simulation à {time} !")

            yield env.timeout(time_before_next)
            
            # Detecter quelle heure est-t-il
            if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour != current_hour:
                i += 1 
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour

        else:
            setup = True
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
            rt = (timestampnow+total_sec(nb_jours))-env.now
            if rt <= 1:
                print(f"Fin de la simulation à {time} !")
            yield env.timeout(1) 

In [55]:
# Constante 
nb_days = 30
prix_per_imp = 1
Budget_par_jour = 3000

In [56]:
timestampnow = int(time.time()-3600*12)
# On génère un nouveau name tuple qui prend en compte les variables d'achat, le budget dépensé, le budget par seconde etc
bidrequests =  namedtuple(
    "bidrequests", 
    (
        "identifiant", 
        "timestamp", 
        "timestamp_string",
        "nombre_impressions",
        "prix_impression",
        "prix_total",
        "achat",
        "budget_depense",
        "bt",
        "cible"
    )
                    )
# Liste pour stocker les résultats du namedtuple
data = list()
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb_pacing(env, prix_per_imp, Budget_par_jour, timestampnow, nb_days, 14, bidrequests, data))

In [57]:
env.run(until=timestampnow + total_sec(nb_days))

Fin de la simulation à 07-17-2020 05:50:48 !


In [58]:
sauvegarde(data, '17-06-2020_1month.csv')

### Dépenser tout le budget 

L'algorithme actuel est efficace pour garantir une dépense plutôt uniformisée sur la journée. Cependant en fin de journée, l'algo rencontre beaucoup de difficultés : 

1. S'il n'y a pas assez de bid requests, alors le budget ne sera pas dépensé totalement ce qui est problématique
2. Il y a apparition de gros pics en fin de période que l'on voudrait étaler plutôt sur toute la période

Il faut donc assurer le deuxième objectif de dépenser tout le budget. 

Pour cela, plusieurs idées initiales : 
1. Régles "manuelles" afin d'accélérer la dépense en fin de période 
2. Règles basées sur les proportions d'achat de br -> Si on tend vers 100%, il faut accélérer la dépense
3. Ajuster dynamiquement une estimation du délais entre 2 br (supposé constant dans la journée) afin d'intégrer les phénomènes d'accélération et de ralentissement dans l'arrivée des br

#### Analyse de données historiques

In [67]:
df = pd.read_csv('17-06-2020_1month.csv', index_col="timestamp_string", parse_dates=True)
df.index.names = ['Date']
df.tail()

,identifiant,timestamp,nombre_impressions,prix_impression,prix_total,achat,budget_depense,bt,cible
Date,,,,,,,,,
2020-07-16 19:47:40,110365,1594921660,5,1,5,0,2998,1.0,0.059524
2020-07-16 19:47:47,110366,1594921667,6,1,6,0,2998,1.0,0.059524
2020-07-16 19:47:55,110367,1594921675,1,1,1,1,2999,1.0,0.059524
2020-07-16 19:48:02,110368,1594921682,2,1,2,0,2999,1.0,0.059524
2020-07-16 19:48:04,110369,1594921684,5,1,5,0,2999,1.0,0.059524


Essayons de calculer le budget restant à la fin de chaque jour 

In [68]:
df['budget_restant'] = Budget_par_jour - df['budget_depense']

In [69]:
new_df = df.groupby(df.index.day).tail(1)
new_df

,identifiant,timestamp,nombre_impressions,prix_impression,prix_total,achat,budget_depense,bt,cible,budget_restant
Date,,,,,,,,,,
2020-06-17 19:51:05,4346,1592416265,5,1,5,0,3000,-0.0,0.059524,0
2020-06-18 19:59:57,8656,1592503197,1,1,1,0,3000,-0.0,0.059524,0
2020-06-19 19:51:23,12667,1592589083,8,1,8,0,3000,-0.0,0.059524,0
2020-06-20 19:54:11,15855,1592675651,0,1,0,0,3000,-0.0,0.059524,0
2020-06-21 19:43:39,19419,1592761419,3,1,3,0,3000,-0.0,0.059524,0
2020-06-22 19:44:19,23226,1592847859,2,1,2,0,3000,-0.0,0.059524,0
2020-06-23 19:57:48,26879,1592935068,2,1,2,0,3000,-0.0,0.059524,0
2020-06-24 19:54:35,31938,1593021275,2,1,2,0,3000,-0.0,0.059524,0
2020-06-25 19:57:01,35028,1593107821,0,1,0,0,3000,-0.0,0.059524,0


En réalité dans les simulations, je ne pense pas que le problème provient du fait qu'on ne reçoit pas assez de bid requests dans la fin de journée, mais il provient simplement du fait qu'on estime mal la dernière heure à laquelle on reçoit les br. 

Essayons tout simplement de relancer les simulations de pacing mais en prenant un horaire de fin plus strict que 20h.

On voit que ça marche parfaitement en prenant des horaires plus strictes. Il faudrait après soit refaire le même type d'analyse sur des vraies données, soit changer le cadre de simulation...